In [85]:
# 1) Подразделения
SUBDIVISIONS = [
    'аор',
    'тск',
    'ао кропоткинское',
    'восход',
    'колхоз прогресс',
    'мир',
    'сп коломейцево'
]

# 2) Операции
OPERATION_PATTERNS = [
    (r'предп',                       'Предпосевная культивация'),
    (r'посев|сев',                   'Сев'),
    (r'гербицид|сзр',                'Гербицидная обработка'),
    (r'подкорм',                     'Подкормка'),
    (r'борон',                       'Боронование довсходовое'),
    (r'убор',                        'Уборка'),
    (r'2-я подкорм',                 '2-я подкормка'),
    (r'внесение удобр|внесение мин', 'Внесение минеральных удобрений'),
    (r'чизел',                       'Чизлевание'),
    (r'2-ое диск|2-е диск',          'Дискование 2-е'),
    (r'\bдиск',                      'Дискование'),
    (r'прик',                        'Прикатывание посевов'),
    (r'вырав',                       'Выравнивание зяби'),
    (r'\bпах',                       'Пахота'),
]

# 3) Культуры
CULTURE_PATTERNS = [
    (r'рапс|рап',        'Рапс озимый'),
    (r'овс|овес',        'Овес'),
    (r'мн\s*тр',         'Многолетние травы текущего года'),
    (r'\bпш\b',          'Пшеница озимая товарная'),
    (r'пшен',            'Пшеница озимая товарная'),
    (r'зел\s*корм',      'Пшеница озимая на зеленый корм'),
    (r'ячмен',           'Ячмень озимый'),
    (r'семенн',          'Соя семенная'),
    (r'озим',            'Пшеница озимая товарная'),
    (r'кук(?:/|\s+)сил', 'Кукуруза кормовая'),
    (r'\bкук',           'Кукуруза товарная'),
    (r'св[её]кл|сах\s*св','Свекла сахарная'),
    (r'подс',            'Подсолнечник товарный'),
    (r'со[йю]',          'Соя товарная'),
]

def detect_subdivision(line: str) -> str:
    ll = line.lower()
    for sub in SUBDIVISIONS:
        if sub in ll:
            if sub == 'аор': return 'АОР'
            if sub == 'тск': return 'ТСК'
            return sub.capitalize()
    return ""

def detect_date_and_year(line: str):
    dm = re.search(r"\b(\d{1,2}\.\d{1,2})\b", line)
    yr = re.search(r"\b(20\d{2})\s*г?\b", line)
    return (dm.group(1) if dm else "", yr.group(1) if yr else "")

def combine_date_with_year(dmy: str, year: str) -> str:
    if dmy and not year:
        year = str(datetime.today().year)
    if dmy and year:
        d, m = dmy.split('.')
        return f"{int(d):02d}.{int(m):02d}.{year}"
    return dmy

def detect_operation(text: str) -> str:
    t = text.lower()
    for pat, name in OPERATION_PATTERNS:
        if re.search(pat, t):
            return name
    return ""

def detect_cultures(text: str):
    clean = text.replace('.', ' ').replace('-', ' ')
    found = []
    for pat, name in CULTURE_PATTERNS:
        for m in re.finditer(pat, clean):
            found.append((m.start(), name))
    found.sort(key=lambda x: x[0])
    return found  # list of (pos, culture)

def parse_areas_in_text(text: str):
    found = []
    # 1) пары X/YYY
    for m in re.finditer(r"(\d+)\s*/\s*(\d+)", text):
        found.append((m.start(), int(m.group(1)), int(m.group(2))))
    if found:
        found.sort(key=lambda x: x[0])
        return found
    # 2) одиночные X га (не после '/')
    for m in re.finditer(r"(\d+)\s*га\b", text):
        if m.start() and text[m.start() - 1] == '/':
            continue
        found.append((m.start(), int(m.group(1)), int(m.group(1))))
    found.sort(key=lambda x: x[0])
    return found

def parse_agro_message(message: str):
    # 1) Дата и подразделение
    cur_date = cur_year = ""
    cur_sub = "АОР"
    for ln in message.splitlines():
        if not ln.strip(): continue
        dmy, yr = detect_date_and_year(ln)
        if dmy and not cur_date: cur_date = dmy
        if yr  and not cur_year: cur_year = yr
        sd = detect_subdivision(ln)
        if sd: cur_sub = sd
    final_date = combine_date_with_year(cur_date, cur_year)

    # 2) Разбиваем на блоки по операциям
    blocks, current = [], []
    for ln in message.splitlines():
        if detect_operation(ln):
            if current: blocks.append(current)
            current = [ln]
        else:
            current.append(ln)
    if current: blocks.append(current)

    # 3) Парсим каждый блок
    results = []
    for block in blocks:
        op_name = detect_operation(block[0])
        raw = " ".join(block).lower()
        text = raw.replace('.', ' ').replace('-', ' ')

        # Сначала — если есть "по пу X/YYY", берём только его
        pu = re.search(r"по\s*пу\s*(\d+)\s*/\s*(\d+)", text)
        if pu:
            areas = [(pu.start(), int(pu.group(1)), int(pu.group(2)))]
        else:
            areas = parse_areas_in_text(text)

        cultures = detect_cultures(text)

        for pos, day, total in areas:
            # выбираем последнюю культуру до позиции
            cult = ""
            for cpos, cname in cultures:
                if cpos <= pos:
                    cult = cname
                else:
                    break
            results.append({
                "Дата": final_date,
                "Подразделение": cur_sub,
                "Операция": op_name,
                "Культура": cult,
                "За день, га": day,
                "С начала операции, га": total,
                "Вал за день, ц": "",
                "Вал с начала, ц": ""
            })

    return results

In [86]:
msg = """Восход
Посев кук-24/252га
24%
Предпосевная культ
Под кук-94/490га46%
СЗРоз пш-103/557га
25%
Подкормка оз рапс -
152га , 100%, подкормка овса-97га, 50%
Довсходовое боронование подсолнечника-524
га, 100%.
"""
import pandas as pd
df = pd.DataFrame(parse_agro_message(msg))
display(df)

,Дата,Подразделение,Операция,Культура,"За день, га","С начала операции, га","Вал за день, ц","Вал с начала, ц"
0,,Восход,Сев,Кукуруза товарная,24,252,,
1,,Восход,Предпосевная культивация,Кукуруза товарная,94,490,,
2,,Восход,Гербицидная обработка,Пшеница озимая товарная,103,557,,
3,,Восход,Подкормка,,152,152,,
4,,Восход,Подкормка,Овес,97,97,,
5,,Восход,Боронование довсходовое,Подсолнечник товарный,524,524,,


In [53]:
messages = [
    """Пахота зяби под мн тр
По Пу 26/488
Отд 12 26/221

Предп культ под оз пш
По Пу 215/1015
Отд 12 128/317
Отд 16 123/529

2-е диск сах св под пш
По Пу 22/627
Отд 11 22/217

2-е диск сои под оз пш
По Пу 45/1907
Отд 12 45/299""",

    """Пахота зяби под сою 
По ПУ 7/1402
Отд 17 7/141

Вырав-ие зяби под кук/силос
По ПУ 16/16
Отд 12 16/16

Вырав-ие зяби под сах/свёклу
По ПУ 67/912
Отд 12 67/376

2-ое диск-ие сах/свёкла 
По ПУ 59/1041
Отд 17 59/349""",

    """12.10
Внесение мин удобрений под оз пшеницу 2025 г ПУ Юг 149/7264
Отд 17-149/1443""",

    """Север 
Отд7 пах с св 41/501
Отд20 20/281 по пу 61/793
Отд 3 пах подс.60/231
По пу 231

Диск к. Сил отд 7. 32/352
Пу- 484
Диск под Оз п езубов 20/281
Диск под с. Св отд 10 83/203 пу-1065га""",

    """Внесение удобрений под рапс отд 7 -138/270
Дисклвание под рапс 40/172
Диск после Кук сил отд 7-32/352 по пу 484га""",

    """10.03 день
2-я подкормка озимых, ПУ "Юг" - 1749/2559
(в т.ч Амазон-1082/1371
Пневмоход-667/1188)

Отд11- 307/307 (амазон 307/307) 

Отд 12- 671/671( амазон 318/318; пневмоход 353/353) 

Отд 16- 462/1272( амазон 148/437; пневмоход 314/835) 

Отд 17- 309/309( амазон 309/309)""",

    """Уборка свеклы 27.10.день
Отд10-45/216
По ПУ 45/1569
Вал 1259680/6660630
Урожайность 279,9/308,3
По ПУ 1259680/41630600
На завод 1811630/6430580
По ПУ 1811630/41400550
Положено в кагат 399400
Вввезено с кагата 951340
Остаток 230060
Оз-9,04/12,58
Дигестия-14,50/15,05""",

    """Пахота под сах св
По Пу 77/518
Отд 12 46/298
Отд 16 21/143
Отд 17 10/17

Чизел под оз ячмень 
По Пу 22/640
Отд 11 22/242

Чизел под оз зел корм
Отд 11 40/40

Диск оз пшеницы
По Пу 28/8872
Отд 17 28/2097

2-е диск под сах св
По Пу 189/1763
Отд 11 60/209
Отд 12 122/540
Отд 17 7/172

Диск кук силос
По Пу 6/904
Отд 11 6/229

Прик под оз ячмень
По Пу 40/498
Отд 11 40/100

Уборка сои (семенной)
Отд 11 65/65
Вал 58720
Урож 9""",

    """Пахота под сах св
ПоПу 91/609
Отд 11 13/73
Отд 12 49/347
Отд 16 20/163
Отд 17 9/26

Чизел под оз зел корм
Отд 11 60/100

2-е диск под сах св
По Пу 53/1816
Отд 12 53/593

Диск кук силос
По Пу 66/970
Отд 11 66/295

Диск сах св
Отд 12 13/13

Прикат под оз ячмень
По Пу 40/538
Отд 11 40/140

Уборка сои семенной
Отд 11 29/94
Вал 37400/96120
Урож 12,9/10,2""",

    """Пахота под сах св
По Пу 88/329
Отд 11 23/60
Отд 12 34/204
Отд 16 31/65

Пахота под мн тр
По Пу 10/438
Отд 17 10/80

Чизел под оз ячмень
По Пу 71/528
Отд 11 71/130

2-е диск под сах св
По Пу 80/1263
Отд 12 80/314

2-е диск под оз ячмень
По Пу 97/819
Отд 17 97/179

Диск кук силос
По Пу 43/650
Отд 11 33/133
Отд 12 10/148

Выкаш отц форм под/г
Отд 12 10/22

Уборка сах св
Отд 12 16/16
Вал 473920
Урож 296,2
Диг - 19,19
Оз - 5,33""",

    """20.11 Мир
Пахота зяби под сою 100 га день, 1109 га от начала, 97%, 30 га остаток.
Работало 4 агрегата.
Выравнивание зяби под подсолнечник 47 га день, 141 га от начала, 29 %, остаток 565 га. Работал 1 агрегат

Осадки:
Бригада 1 Воронежская - 6 мм""",

    """ТСК
Выравнивание зяби под сою 25 га/ с нарастающим 765 га ( 13%) Остаток 5332 га
Выравнивание зяби под кукурузу 131га (3%) Остаток 4486 га
Осадки 1мм""",

    """Восход
Посев кук-24/252га
24%
Предпосевная культ
Под кук-94/490га46%
СЗРоз пш-103/557га
25%
Подкормка оз рапс-
152га , 100%, подкормка овса-97га, 50%
Довсходовое боронование подсолнечника-524
га, 100%.""",

    """30.03.25г.
СП Коломейцево

предпосевная культивация  
  -под подсолнечник
    день 30га
    от начала 187га(91%)

сев подсолнечника 
  день+ночь 57га
  от начала 157га(77%)

Внесение почвенного гербицида по подсолнечнику 
  день 82га 
  от начала 82га (38%)"""
]


In [87]:
for i, msg in enumerate(messages, start=1):
    print(f'Сообщение №{i}')
    df = pd.DataFrame(parse_agro_message(msg))
    display(df)


Сообщение №1


,Дата,Подразделение,Операция,Культура,"За день, га","С начала операции, га","Вал за день, ц","Вал с начала, ц"
0,,АОР,Пахота,Многолетние травы текущего года,26,488,,
1,,АОР,Предпосевная культивация,Пшеница озимая товарная,215,1015,,
2,,АОР,Дискование 2-е,Пшеница озимая товарная,22,627,,
3,,АОР,Дискование 2-е,Пшеница озимая товарная,45,1907,,


Сообщение №2


,Дата,Подразделение,Операция,Культура,"За день, га","С начала операции, га","Вал за день, ц","Вал с начала, ц"
0,,АОР,Пахота,Соя товарная,7,1402,,
1,,АОР,Выравнивание зяби,Кукуруза товарная,16,16,,
2,,АОР,Выравнивание зяби,Свекла сахарная,67,912,,
3,,АОР,Дискование 2-е,Свекла сахарная,59,1041,,


Сообщение №3


...

,Дата,Подразделение,Операция,Культура,"За день, га","С начала операции, га","Вал за день, ц","Вал с начала, ц"
0,,ТСК,Выравнивание зяби,Соя товарная,25,25,,
1,,ТСК,Выравнивание зяби,Соя товарная,765,765,,
2,,ТСК,Выравнивание зяби,Соя товарная,5332,5332,,
3,,ТСК,Выравнивание зяби,Кукуруза товарная,131,131,,
4,,ТСК,Выравнивание зяби,Кукуруза товарная,4486,4486,,


Сообщение №13


,Дата,Подразделение,Операция,Культура,"За день, га","С начала операции, га","Вал за день, ц","Вал с начала, ц"
0,,Восход,Сев,Кукуруза товарная,24,252,,
1,,Восход,Предпосевная культивация,Кукуруза товарная,94,490,,
2,,Восход,Гербицидная обработка,Пшеница озимая товарная,103,557,,
3,,Восход,Подкормка,,152,152,,
4,,Восход,Подкормка,Овес,97,97,,
5,,Восход,Боронование довсходовое,Подсолнечник товарный,524,524,,


Сообщение №14


,Дата,Подразделение,Операция,Культура,"За день, га","С начала операции, га","Вал за день, ц","Вал с начала, ц"
0,30.03.2025,Сп коломейцево,Предпосевная культивация,Подсолнечник товарный,30,30,,
1,30.03.2025,Сп коломейцево,Предпосевная культивация,Подсолнечник товарный,187,187,,
2,30.03.2025,Сп коломейцево,Сев,Подсолнечник товарный,57,57,,
3,30.03.2025,Сп коломейцево,Сев,Подсолнечник товарный,157,157,,
4,30.03.2025,Сп коломейцево,Гербицидная обработка,Подсолнечник товарный,82,82,,
5,30.03.2025,Сп коломейцево,Гербицидная обработка,Подсолнечник товарный,82,82,,
